In [1]:
from src.search.paragraphs import DocumentStore, Document
from src.search.passage_ranking import ResponseScorer

2022-06-08 09:46:12.149039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/lib:/usr/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/opt/amazon/efa/lib:/usr/local/mpi/lib:
2022-06-08 09:46:12.149072: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
host = '<es-host>'

In [3]:
index = 'tmp_es_template' # you have to change the index name to something like '<your-firstname>-blahblah'

In [4]:
storeclient = DocumentStore(host)

In [5]:
storeclient.create_store(index, lang='fr', reset_index=True)

INFO:elasticsearch:DELETE http://172.31.18.19:9200/tmp_es_template [status:200 request:0.035s]
INFO:elasticsearch:PUT http://172.31.18.19:9200/tmp_es_template [status:200 request:0.097s]


In [6]:
with open('sample.txt') as f:
    data = [Document(text=s.strip(), id=i) for i, s in enumerate(f.readlines())]

In [7]:
data

[Document(text="Quel est le capital minimum d'une SA ?", id=0),
 Document(text='Qu’est-ce qu’un contrat de capitalisation ?', id=1)]

In [8]:
storeclient.add_documents(index=index, documents=data)

INFO:elasticsearch:POST http://172.31.18.19:9200/_bulk?refresh=true [status:200 request:0.011s]


In [9]:
query = 'un contrat de capitalisation'
#query = 'mediteranen'
#query = 'civilizatio'

In [10]:
print('search results:')
print(storeclient.search(index, query))
storeclient.suggest(index=index, query_string=query)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.004s]
INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.002s]


search results:
[SearchResult(paragraph=Document(text='Qu’est-ce qu’un contrat de capitalisation ?', id='1'), highlight=['Qu’est-ce qu’un <b>contrat</b> de <b>capitalisation</b> ?'], score=0.8754687), SearchResult(paragraph=Document(text="Quel est le capital minimum d'une SA ?", id='0'), highlight=["Quel est le <b>capital</b> minimum d'une SA ?"], score=0.18232156)]


([{'text': 'un', 'offset': 0, 'length': 2, 'options': []},
  {'text': 'contrat', 'offset': 3, 'length': 7, 'options': []},
  {'text': 'de', 'offset': 11, 'length': 2, 'options': []},
  {'text': 'capitalisation', 'offset': 14, 'length': 14, 'options': []}],
 [{'text': 'contrat', 'offset': 3, 'length': 7, 'options': []},
  {'text': 'capital', 'offset': 14, 'length': 14, 'options': []}])

In [11]:
scorer = ResponseScorer()

Some weights of the model checkpoint at models/asnq_roberta were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 14.56 GiB total capacity; 278.47 MiB already allocated; 5.50 MiB free; 288.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
storeclient.search(index, query, ranker=scorer)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.003s]
INFO:root:Score passages...
INFO:root:Prepare data: 0.00s
100%|██████████| 1/1 [00:00<00:00, 62.46it/s]
INFO:root:Inference: 0.06


[SearchResult(paragraph=Document(text="Napoleon Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history.[3][4]", id='0'), highlight=['<b>Napoleon</b> Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the'], score=0.4489596486091614)]

In [ ]:
[ doc1, doc2, ..., docn ] -> elasticsearch's index -> query
es:
- use tf-idf (bm25) -> [ doc_i1, ..., doc_ij ]
- use QA model: (query, doc) -> 0 <= score <= 1
    [(doc_i1, sc1), ..., (doc_ij, scj)]